In [41]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from denoicer import Denoicer
from mecab_wakati import MecabWakati
from mbti_util import MbtiUtil

In [42]:
denoicer = Denoicer()
mecab_wakati = MecabWakati()
mbti_util = MbtiUtil()

In [48]:
def read_tweets_with_before_after():
    path = './database/tweets_with_before_or_after.tsv'
    df = pd.read_csv(path, sep='\t')
    # df = df[df['tweet_text'].str.match('(?!@(\w+) )')]
    return df

df = read_tweets_with_before_after()
df.head(5)

,Unnamed: 0,user_id,m_type,tweet_id,tweet_text,m_type_en,before_or_after
0,0,940954542576885760,仲介者,1197852103021981696,@yryr_8945 担当制なんだ！笑この量でこのクオリティは本当すごい🎅お疲れ様です☺️,INFP,before
1,1,940954542576885760,仲介者,1197318394946543617,@yryr_8945 えっ！自作！すごい！,INFP,before
2,2,940954542576885760,仲介者,1196803484332388352,@stella_kakerun 明日休みだから行ってみる😭ありがとう！,INFP,before
3,3,940954542576885760,仲介者,1196785937482575874,ファミマ行く暇なかったよー😭,INFP,before
4,4,940954542576885760,仲介者,1196389448344207370,シュヴァルツヴェルダーキルシュトルテかな？ https://t.co/ggxZQ2v6hL,INFP,before


In [49]:
def create_user_words(m_type, before_or_after):
    df_copy = df[df['m_type_en'] == m_type]
    user_ids = list(set(df_copy['user_id']))
    words = []
    for ids in user_ids:
        df_copy = df_copy[df_copy['user_id'] == ids]
        df_copy = df_copy[df_copy['before_or_after'] == before_or_after]
        user_words = []
        for tweet in df_copy['tweet_text']:
            user_words += mecab_wakati.wakati_sentence(denoicer.normalize_text(tweet))
        words += list(user_words)
    return words


docs = []
for m_type in sorted(mbti_util.m_types):
    docs.append(' '.join(create_user_words(
        m_type=m_type, before_or_after="after")))


In [50]:
vectorizer = TfidfVectorizer(max_df=0.7)  # 文書全体の90%以上で出現する単語は無視する
X = vectorizer.fit_transform(docs)
values = X.toarray()
feature_names = vectorizer.get_feature_names()

/mnt/data1/home/kishima/research-covid-19-mbti/venv/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [51]:
output = []
for m_type, vec in zip(sorted(mbti_util.m_types), values):
    output.append(f'{m_type}\n')
    for w_id, tfidf in sorted(enumerate(vec), key=lambda x: x[1], reverse=True)[:20]:
        word = feature_names[w_id]
        output.append(f'{word}\t{tfidf}\n')

with open('result.txt', 'w', encoding='utf-8') as f:
    for o in output:
        f.write(o)

In [55]:
# データフレームからある単語を含むツイート文の行を抜き出す（現在は「生きる」が含まれる行を抽出）
# コロナ禍前後か否かとMBTIのタイプを指定
before_after = "after"
m_type = "ISFP"
with open('result1.txt', 'w', encoding='utf-8') as f:
    tmp = df.query(
        'tweet_text.str.contains("生きる")', engine='python')[df['m_type_en'] == m_type][df['before_or_after'] == before_after]
    for i, j in zip(tmp['user_id'], tmp['tweet_text']):
        f.write(f'{i}\t{j}\n')


/tmp/ipykernel_1822412/1217411716.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tmp = df.query(
